# Applicative

## Outline

* Incentive for Applicative

* Using Applicative and Functor together

* Applicative examples

* Applicative laws

In this lesson, we will learn about the Applicative type classe and how you can use it.

## Incentive for Applicative

Imagine you have a function `maybeAdd1 :: Num a => Maybe (a -> a)`. This means that the function migh exist or not. How can this be? Well a practical example is if you use the `<$>` operator with a function like `(+)` and apply it to a `Maybe a` value. Here is a simple example.

In [ ]:
maybeAddition :: Num a => Maybe a -> Maybe (a -> a)
maybeAddition maybeVal = (+) <$> maybeVal

-- Here is a more detailed breakdown of this function without using <$>
maybeAddition' :: Num a => Maybe a -> Maybe (a -> a)
maybeAddition' Nothing = Nothing
maybeAddition' (Just n) = Just $ (+) n

var1 :: Maybe Int
var1 = Just 1

maybeAdd1 :: Maybe (Int -> Int)
maybeAdd1 = maybeAddition var1
:t maybeAdd1

We see that the command :t in the code above returns the signature we defined in the begining where type a is now Int. Now comes the question if you want to use this function on another `Maybe Int` variale how would you do it? One possibility would be to write a custom function that handles the case of the missing function.

In [ ]:
import Data.Maybe ( fromJust )

var2 :: Maybe Int
var2 = Just 2

add1 :: Maybe Int -> Maybe Int
add1 Nothing = Nothing
add1 (Just n) = Just $ fromJust maybeAdd1 n

print $ add1 var2

Another option is to use the functions the Applicative type class provides:
```haskell
class Functor f => Applicative f where
  pure :: a -> f a
  (<*>) :: f (a -> b) -> f a -> f b
  GHC.Base.liftA2 :: (a -> b -> c) -> f a -> f b -> f c
  (*>) :: f a -> f b -> f b
  (<*) :: f a -> f b -> f a
```

We see that Functor is a superclass of Applicative. The minimal complete definition requires just the app operator `<*>` or `liftA2`. Since the Maybe type has also an instance of Applicative let's see how the app operator can help us to avoid using again custom defined functions for transformations. Here is the same problem solved with `<*>`.

In [ ]:
add1UsingApp :: Maybe Int -> Maybe Int
add1UsingApp maybeInt = maybeAdd1 <*> maybeInt

print $ add1UsingApp var2

We get the same result as before and the function is one line shorter. But this is not main reason why to use this operator. The main reason is that it's easier to read the code because it's generic. The same symbol serves the same purpose for all Applicatives. 

## Using Applicative and Functor together

If we think a bit about the examples from the previous chapter we may ask ourselves how to write a function `addUsingApp` that takes in two Maybe values and returns also a Maybe value.

In [ ]:
addTwoUsingApp :: Num a => Maybe a -> Maybe a -> Maybe a
addTwoUsingApp maybeVal1 maybeVal2 = (+) <$> maybeVal1 <*> maybeVal2

print $ addTwoUsingApp var1 var2

We see we get again the same result and used much less lines of code. Because of how `<*>` is defined you can always add one more parameter to the function. Also instead of using the `<$>` operator you can use `pure` that puts the function in the begining directly in the maybe context.

In [ ]:
var3 :: Maybe Int
var3 = Just 3

addThreeUsingApp :: Num a => Maybe a -> Maybe a -> Maybe a -> Maybe a
addThreeUsingApp maybeVal1 maybeVal2 maybeVal3 = pure (+) <*> (pure (+) <*> maybeVal1 <*> maybeVal2) <*> maybeVal3

print $ addThreeUsingApp var1 var2 var3

## Applicative examples

Another way how to use the Applicative type class is in IO which has also an instance of Applicative. Let's imagine you ask the user for two numbers and then calculate their product. Here is the code where we make use of the `<*>` and `<$>` operators.

In [ ]:
myProduct :: Num a => a -> a -> a
myProduct x1 x2 = x1*x2

readDouble :: IO Double
readDouble = read <$> getLine

intProduct :: IO Double
intProduct = myProduct <$> readDouble <*> readDouble

main :: IO ()
main = do
    print "Input 2 numbers. Use dot for decimal seperator."
    result <- intProduct
    print result

main

You can also use the `<*>` operator on lists to get all possible combinations. This is because if you view a list type as a context instead as a container, you can say that a list represents more possible values. For instance [Int] represent more possible values of type Int. Here is a code example:

In [ ]:
list1 = [1,2,3] :: [Int]
list2 = [4,5] :: [Int]

allCombinations :: Num a => [a] -> [a] -> [a]
allCombinations l1 l2 = pure (+) <*> l1 <*> l2

print $ allCombinations list1 list2

Another example for creating all possible combination is by using lists to create an instance of a user defined type with record syntax. Here is the code:

In [ ]:
data Car = Car {
    company :: String,
    color :: String
} deriving Show

companies = ["Toyota", "Mercedes", "Ford"]
colors = ["yellow", "gree", "blue"]

allPossibleCars :: [Car]
allPossibleCars = pure Car <*> companies <*> colors

print allPossibleCars

## Applicative laws

Applicative has 4 laws. They are as follows:

- Identity:<br>`pure id <*> x = x`

- Composition:<br>`pure (.) <*> x1 <*> x2 <*> x3 = x1 <*> (x2 <*> x3)`

- Homomorphism:<br>`pure f <*> pure x = pure (f x)`

- Interchange:<br>`x1 <*> pure x2 = pure ($ x2) <*> x1`

## Recap

In this lesson we've discussed:

- the motivation for introducing the Applicative type 

- how to use the `<*>` operator together with `<$>`

- examples that show how to use the Applicative type class

- laws that apply to the Applicative type class